# Analyse du dataset UCIML Adult


In [28]:
import pandas as pd
from ucimlrepo import fetch_ucirepo
from os.path import join

In [29]:
adult = fetch_ucirepo(name="Adult")

X = adult.data.features
y = adult.data.targets

In [30]:
df = adult.data.original
df.to_csv(join("..", "data", "raw", "adult.csv"))

In [31]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

## Analyse des valeurs cibles


In [32]:
y.shape

(48842, 1)

In [33]:
y.describe()

,income
count,48842
unique,4
top,<=50K
freq,24720


### Exploration des valeurs uniques


In [34]:
df.income.value_counts()

income
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
Name: count, dtype: int64

### Gestion des labels erronés

Alors que les valeurs cibles ne devraient comprendre que 2 labels, on en compte ici 4. Cela s'explique par la duplication des labels avec un point supplémentaire en fin de chaîne. Il est indispensable de corriger cette erreur de labélisation pour préserver les 2 catégories souhaitées.


In [35]:
df.loc[:, "income"] = df["income"].replace({"<=50K.": "<=50K", ">50K.": ">50K"})
df.income.value_counts()

income
<=50K    37155
>50K     11687
Name: count, dtype: int64

## Analyses des features


In [36]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


In [37]:
X.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


### Analyse éthique


In [38]:
X.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')

#### Columns analysis

- `race`: Utiliser la race comme prédicteur de revenu risque de perpétuer des biais systémiques et est illégal dans de nombreux contextes.
- `sex`: Utiliser le genre comme prédicteur de revenu risque de perpétuer les inégalités salariales homme-femme.
- `native-country`: Le pays d'origine peut avoir un impact sur le revenu. La barrière de la langue, l'éducation différente et la reconnaissance des diplômes d'un pays à l'autre sont autant de facteurs qui lient le pays de naissance au revenu. Cependant, cette donnée pourrait perpétuer des discriminations basées sur l'origine. De plus, il est illégal de conserver cette donnée en Europe.

- `marital-status` et `relationship` peuvent avoir une influence non négligeable sur les revenus. Cependant, ces colonnes risques de


In [39]:
unethical_cols = ["race", "sex", "native-country"]
# questionnable_cols = ["marital-status", "relationship"]
questionnable_cols = ["marital-status"]
useless_cols = ["fnlwgt"]
cols_to_drop = unethical_cols + questionnable_cols + useless_cols

ethical_df = df.drop(columns=cols_to_drop)

In [40]:
ethical_df.columns

Index(['age', 'workclass', 'education', 'education-num', 'occupation',
       'relationship', 'capital-gain', 'capital-loss', 'hours-per-week',
       'income'],
      dtype='object')

In [41]:
cols_with_empty_values = ["workclass", "occupation"]
total_lines_count = df.shape[0]

for col in cols_with_empty_values:
    empty_line_count = df[col].isnull().sum()
    empty_line_percentage = empty_line_count / total_lines_count * 100
    print(f"{col} empty lines: {empty_line_count} ({empty_line_percentage:.2f}%)")

print("-" * 40)
total_empty_line_count = df.isnull().any(axis=1).sum()
total_empty_line_percentage = total_empty_line_count / total_lines_count * 100
print(f"total empty lines: {empty_line_count} ({total_empty_line_percentage:.2f}%)")

workclass empty lines: 963 (1.97%)
occupation empty lines: 966 (1.98%)
----------------------------------------
total empty lines: 966 (2.50%)


Les lignes vides ne représentent que 2,50% des données. Je décide de les supprimer.

In [42]:
processed_df = ethical_df.dropna()
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47876 entries, 0 to 48841
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             47876 non-null  int64 
 1   workclass       47876 non-null  object
 2   education       47876 non-null  object
 3   education-num   47876 non-null  int64 
 4   occupation      47876 non-null  object
 5   relationship    47876 non-null  object
 6   capital-gain    47876 non-null  int64 
 7   capital-loss    47876 non-null  int64 
 8   hours-per-week  47876 non-null  int64 
 9   income          47876 non-null  object
dtypes: int64(5), object(5)
memory usage: 4.0+ MB


In [43]:
processed_df.to_csv(join("..", "data", "processed", "adult.csv"))